<a href="https://colab.research.google.com/github/leedhn/capstone21-1/blob/main/make_pickle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
import pandas as pd
import os
import numpy as np
import torch

### load data

In [ ]:
pickle_file = pd.read_pickle(r'/content/drive/MyDrive/21-1_SoJong/20210407/path_embedding.pickle')

In [ ]:
# check tensor shape
pickle_file['train'].shape, pickle_file['test'].shape, pickle_file['dev'].shape

(torch.Size([9741, 5, 5, 768]),
 torch.Size([1140, 5, 5, 768]),
 torch.Size([1221, 5, 5, 768]))

In [ ]:
# 대체할 애들 변수 선언
replacement1 = pickle_file['train'].view(243525, -1)[243] # _isa 어쩌구 path의 embedding 값 _isa 어쩌구 path의 embedding 값
replacement2 = torch.zeros(768) # 그냥 zero

In [ ]:
# read mnli result
result_dir = '/content/drive/MyDrive/21-1_SoJong/20210407/mnli_result'
mnli_train = pd.read_csv(os.path.join(result_dir, 'except_dump_train_result.csv')).values
mnli_dev = pd.read_csv(os.path.join(result_dir, 'except_dump_dev_result.csv')).values
mnli_test = pd.read_csv(os.path.join(result_dir, 'except_dump_test_result.csv')).values

In [ ]:
# check mnli result shape
mnli_train.shape, mnli_test.shape, mnli_dev.shape

((243525, 1), (28500, 1), (30525, 1))

In [ ]:
# check : _is a의 이상한 path들끼리는 pickle값 같은가?
# pickle['train'].view(243525, -1)[243] == pickle['train'].view(243525, -1)[249] # all true : 같다.

In [ ]:
# mnli 결과 확인
pd.DataFrame(mnli_train).value_counts(), pd.DataFrame(mnli_test).value_counts(), pd.DataFrame(mnli_dev).value_counts()

( 2    119150
  1     69391
  0     34135
 -1     20849
 dtype: int64,  2    13805
  1     8310
  0     3863
 -1     2522
 dtype: int64,  2    14923
  1     8489
  0     4223
 -1     2890
 dtype: int64)

### make new pickle file

In [ ]:
# return : 1) pickle과 같은 모양의 tensor (type : tensor.Tensor) 2) list of tensors (아예 deleted) (type : list)
def replace(pickle, mnli_result, to_delete): #e.g. pickle[train], train, [0, 1] to eliminate contradiction & neutral
  delete_idx = to_delete + [-1] # weird path도 포함해준다
  print(pickle.shape)
  pickle = pickle.view(len(mnli_result), -1)
  print(pickle.shape)
  replaced1, replaced2, deleted = [], [], [] #replaced : _is a어쩌구로 embdding 채운 것, deleted : weird path이거나(-1) 걸러낼 것이면 아예 삭제
  
  for embedding, mnli in zip(pickle, mnli_result):
    if mnli in delete_idx: # replaced1, 2에만 대체해서 추가해줌
      replaced1.append(replacement1)
      replaced2.append(replacement2)
      continue
    replaced1.append(embedding)
    replaced2.append(embedding)
    deleted.append(embedding)
    
  print(len(replaced1), len(deleted), len(replaced2))
  return torch.stack(replaced1).view((-1, 5, 5, 768)), torch.stack(replaced2).view((-1, 5, 5, 768)), deleted

In [ ]:
# 테스트 해보자
r1, r2, d = replace(pickle_file['train'], mnli_train, [0])
type(r1), type(d)

torch.Size([9741, 5, 5, 768])
torch.Size([243525, 768])
243525 188541 243525


(torch.Tensor, list)

In [ ]:
r2.view(-1, 768)[2] # zeros가 나와야 함

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [ ]:
# make new pickle
save_file1 = '/content/drive/MyDrive/21-1_SoJong/20210407/mnli_pickle/neutral/new_path_embedding.pickle'
save_file2 = '/content/drive/MyDrive/21-1_SoJong/20210407/mnli_pickle/neutral/new_path_embedding_zeros.pickle'

path_embeddings_dict1, path_embeddings_dict2 = {}, {}
path_embeddings_dict1['train'],path_embeddings_dict2['train'], _ = replace(pickle_file['train'], mnli_train, [0,1])
path_embeddings_dict1['dev'],path_embeddings_dict2['dev'], _ = replace(pickle_file['dev'], mnli_dev, [0,1])
path_embeddings_dict1['test'],path_embeddings_dict2['test'], _ = replace(pickle_file['test'], mnli_test, [0,1])

with open(save_file1, 'wb') as handle:
    pickle.dump(path_embeddings_dict1, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(save_file2, 'wb') as handle:
    pickle.dump(path_embeddings_dict2, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Check pickle file

In [ ]:
picklepickle = pd.read_pickle(r'/content/drive/MyDrive/21-1_SoJong/20210407/mnli_pickle/neutral/new_path_embedding.pickle')

In [ ]:
picklepickle['train'].shape

torch.Size([9741, 5, 5, 768])

In [ ]:
picklepickle2 = pd.read_pickle(r'/content/drive/MyDrive/21-1_SoJong/20210407/mnli_pickle/neutral/new_path_embedding_zeros.pickle')

In [ ]:
mnli_train[:20]

array([[1],
       [1],
       [0],
       [1],
       [2],
       [1],
       [2],
       [1],
       [2],
       [2],
       [1],
       [2],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1]])

In [ ]:
picklepickle2['train'].view(-1, 768)[0] == picklepickle2['train'].view(-1, 768)[2]

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, Tr

In [ ]:
picklepickle['train'].view(-1, 768)[2] == picklepickle['train'].view(-1, 768)[16] # all True : contradiction이었던 애들은 똑같음

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, Tr

In [ ]:
picklepickle['train'].view(-1, 768)[2] == replacement1

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, Tr

In [ ]:
picklepickle2['train'].view(-1, 768)[2] == replacement2

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, Tr